In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

temp = []
# Load the CSV file
data = pd.read_csv('/kaggle/input/gait-analysis/gait_features_cnn_trained.csv')

# Split the data into training and testing sets, while maintaining the ratio of labels in both sets
train, test = train_test_split(data, test_size=0.2, stratify=data['label'])

# Save the training and testing sets to CSV files
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

df_anomalies = pd.read_csv('/kaggle/input/gait-analysis/gait_features_cnn_nottrained.csv')
train, test = train_test_split(df_anomalies, test_size=0.2, stratify=df_anomalies['label'])
train.to_csv('train1.csv', index=False)
test.to_csv('test1.csv', index=False)

# Split features and labels for trained data
X_trained = data.iloc[:, :-1]
y_trained = pd.Series([1] * len(data)) # Set labels to 1 (trained)

# Split features and labels for anomalies data
X_anomalies = df_anomalies.iloc[:, :-1]
y_anomalies = pd.Series([-1] * len(df_anomalies)) # Set labels to -1 (anomalies)

# Concatenate the dataframes together
df_combined = pd.concat([X_trained, X_anomalies])
y_combined = pd.concat([y_trained, y_anomalies], ignore_index=True)

# Shuffle the rows of the combined dataframe
df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)

# Scale the feature values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_combined)

# Train the Isolation Forest model
iforest = IsolationForest(n_estimators=100, contamination=0.1, random_state=42)
iforest.fit(X_scaled, y_combined)

# Load the new data to predict
df_new = pd.read_csv('/kaggle/input/gait-analysis/Test2.csv')

# Split features and labels
X_new = df_new.iloc[:, :-1]
y_new = df_new.iloc[:, -1]

# Scale the feature values
X_new_scaled = scaler.transform(X_new)

# Predict the scores of the new data
y_pred = iforest.score_samples(X_new_scaled)

# Set a threshold for classifying instances as anomalies
threshold = -0.4
y_pred_class = [1 if y < threshold else 0 for y in y_pred]

# Print the predicted labels
print(y_pred_class)
temp.append(y_pred_class)

df_new = pd.read_csv('/kaggle/input/gait-analysis/test1.csv')

# Split features and labels
X_new = df_new.iloc[:, :-1]
y_new = df_new.iloc[:, -1]

# Scale the feature values
X_new_scaled = scaler.transform(X_new)

# Predict the labels of the new data
y_pred = iforest.predict(X_new_scaled)

# Print the predicted labels
print(y_pred)
temp.append(y_pred.tolist())

print(temp)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM

# Load the CSV file
data = pd.read_csv('/kaggle/input/gait-analysis/gait_features_cnn_trained.csv')
temp1 = []

# Split the data into training and testing sets, while maintaining the ratio of labels in both sets
train, test = train_test_split(data, test_size=0.2, stratify=data['label'])

# Save the training and testing sets to CSV files
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

df_anomalies = pd.read_csv('/kaggle/input/gait-analysis/gait_features_cnn_nottrained.csv')
train, test = train_test_split(df_anomalies, test_size=0.2, stratify=df_anomalies['label'])
train.to_csv('train1.csv', index=False)
test.to_csv('test1.csv', index=False)

# Split features and labels for trained data
X_trained = data.iloc[:, :-1]
y_trained = pd.Series([1] * len(data)) # Set labels to 1 (trained)

# Split features and labels for anomalies data
X_anomalies = df_anomalies.iloc[:, :-1]
y_anomalies = pd.Series([-1] * len(df_anomalies)) # Set labels to -1 (anomalies)

# Concatenate the dataframes together
df_combined = pd.concat([X_trained, X_anomalies])
y_combined = pd.concat([y_trained, y_anomalies], ignore_index=True)

# Shuffle the rows of the combined dataframe
df_combined = df_combined.sample(frac=1, random_state=42).reset_index(drop=True)

# Scale the feature values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_combined)

# Train the One-class SVM model
ocsvm = OneClassSVM(kernel='rbf', nu=0.1, gamma=0.1)
ocsvm.fit(X_scaled, y_combined)

# Load the new data to predict
df_new = pd.read_csv('/kaggle/input/gait-analysis/Test2.csv')

# Split features and labels
X_new = df_new.iloc[:, :-1]
y_new = df_new.iloc[:, -1]

# Scale the feature values
X_new_scaled = scaler.transform(X_new)

# Predict the labels of the new data
y_pred = ocsvm.predict(X_new_scaled)

# Print the predicted labels
print(y_pred)
y_pred[y_pred != 1] = 0
temp1.append(y_pred.tolist())

# Evaluate the model on the test set
y_test = df_new.iloc[:, -1]
y_test[y_test != 1] = 0  # Set non-trained labels to -1
y_pred_test = ocsvm.predict(X_new_scaled)
#print(classification_report(y_test, y_pred_test))


df_new = pd.read_csv('/kaggle/input/gait-analysis/test1.csv')

# Split features and labels
X_new = df_new.iloc[:, :-1]
y_new = df_new.iloc[:, -1]

# Scale the feature values
X_new_scaled = scaler.transform(X_new)

# Predict the labels of the new data
y_pred = ocsvm.predict(X_new_scaled)

# Print the predicted labels
print(y_pred)
y_pred[y_pred != 1] = 0
temp1.append(y_pred.tolist())

# Evaluate the model on the test set
y_test = df_new.iloc[:, -1]
y_test[y_test != 1] = 0  # Set non-trained labels to -1
y_pred_test = ocsvm.predict(X_new_scaled)
print(temp1)

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

result1 = [temp[0] and temp1[0]]
result2 = [temp[1] and temp1[1]]

# Define label encoder and fit on training data
label_encoder = LabelEncoder()
train_data = pd.read_csv('/kaggle/input/gait-analysis/gait_features_cnn_trained.csv')
y_train = train_data['label'].values
label_encoder.fit(y_train)

# Load the trained model
model = load_model('/kaggle/input/cnn-model/gait_classification_model.h5')

if (result1[0] == 1):
    print('Test data 1')
    test_data = pd.read_csv('/kaggle/input/gait-analysis/Test2.csv')

    # Preprocess the test data
    X_test = test_data.drop(['label'], axis=1).values
    X_test = X_test.reshape(-1, 64, 64)
    X_test = np.repeat(X_test[..., np.newaxis], 3, -1)

    # Predict on test data
    y_pred = model.predict(X_test)

    # Inverse transform the predicted labels
    y_pred_labels = label_encoder.inverse_transform(np.argmax(y_pred, axis=1))

    # Print the predicted labels
    print('Predicted Labels:', y_pred_labels)
    
else:
    print('Test data 1 is not of this organization')
    
if (result2[0][0] == 1):
    print('Test data 2')
    test_data = pd.read_csv('/kaggle/input/gait-analysis/test1.csv')

    # Preprocess the test data
    X_test = test_data.drop(['label'], axis=1).values
    X_test = X_test.reshape(-1, 64, 64)
    X_test = np.repeat(X_test[..., np.newaxis], 3, -1)
    
    print('The person as per database is:',test_data['label'].values)

    # Predict on test data
    y_pred = model.predict(X_test)

    # Inverse transform the predicted labels
    y_pred_labels = label_encoder.inverse_transform(np.argmax(y_pred, axis=1))

    # Print the predicted labels
    print('The person identification number as per model is:', y_pred_labels)

else:
    print('Test data 2 is not of this organization')


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

result1 = [temp[0] or temp1[0]]
result2 = [temp[1] or temp1[1]]

# Define label encoder and fit on training data
label_encoder = LabelEncoder()
train_data = pd.read_csv('/kaggle/input/gait-analysis/gait_features_cnn_trained.csv')
y_train = train_data['label'].values
label_encoder.fit(y_train)

# Load the trained model
model = load_model('/kaggle/input/cnn-model/gait_classification_model.h5')

if (result1[0] == 1):
    print('Test data 1')
    test_data = pd.read_csv('/kaggle/input/gait-analysis/Test2.csv')

    # Preprocess the test data
    X_test = test_data.drop(['label'], axis=1).values
    X_test = X_test.reshape(-1, 64, 64)
    X_test = np.repeat(X_test[..., np.newaxis], 3, -1)

    # Predict on test data
    y_pred = model.predict(X_test)

    # Inverse transform the predicted labels
    y_pred_labels = label_encoder.inverse_transform(np.argmax(y_pred, axis=1))

    # Print the predicted labels
    print('Predicted Labels:', y_pred_labels)
    
else:
    print('Test data 1 is not of this organization')
    
if (result2[0] == 1):
    print('Test data 2')
    test_data = pd.read_csv('/kaggle/input/gait-analysis/test1.csv')

    # Preprocess the test data
    X_test = test_data.drop(['label'], axis=1).values
    X_test = X_test.reshape(-1, 64, 64)
    X_test = np.repeat(X_test[..., np.newaxis], 3, -1)
    
    print('The person as per database is:',test_data['label'].values)

    # Predict on test data
    y_pred = model.predict(X_test)

    # Inverse transform the predicted labels
    y_pred_labels = label_encoder.inverse_transform(np.argmax(y_pred, axis=1))

    # Print the predicted labels
    print('The person identification number as per model is:', y_pred_labels)

else:
    print('Test data 2 is not of this organization')
